<a href="https://colab.research.google.com/github/R0b0t-Maker/LLM-T/blob/main/chatbot_LangChain_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pinecone**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r "/content/drive/MyDrive/Udemy/LangChain/p-1/requirements.txt" -q
!pip install --quiet openai python-dotenv
!pip install -q pinecone-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s 

In [ ]:
!pip install --upgrade -q pinecone-client

In [ ]:
!pip show pinecone-client

Name: pinecone-client
Version: 4.1.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 


In [13]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv("/content/drive/MyDrive/Udemy/LangChain/p-1/.env", override=True)

True

In [16]:
from pinecone import Pinecone

pc=Pinecone()
# pc=Pinecone(api_key='e4179379-10db-4ee7-9082-a01a78cda928')
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-apju9rk.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [17]:
pc.list_indexes()[0]

{'dimension': 1536,
 'host': 'langchain-apju9rk.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [18]:
pc.list_indexes()[0]['name']

'langchain'

In [20]:
from pinecone import ServerlessSpec

index_name= 'langchain'
if index_name not in pc.list_indexes().names():
  print(f'Creating index:{index_name}')
  pc.create_index(
      name= index_name,
      dimension=1536,
      metric='cosine',
      spec=ServerlessSpec(
          cloud = 'aws',
          region='us-east-1'
      )

  )
  print('Index created!')
else:
  print(f'Index{index_name} already exsits!')

Creating index:langchain
Index created!


In [21]:
index_name='quickstart'
if index_name in pc.list_indexes().names():
  print(f'Deleting index{index_name}...')
  pc.delete_index(index_name)
  print('done!!!')
else:
  print(f'Index {index_name} does not exist!')

Index quickstart does not exist!


In [22]:
index_name= 'langchain'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [23]:
#inserting vectors
import random

vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_nmae = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [24]:
#updating vectors
index.upsert(vectors=[('c', [0.5]*1536)])
#fetching vectors

index.fetch(ids=['c','d'])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'c': {'id': 'c',
                   'values': [0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
             

In [ ]:
# #deleting vectors
# index.delete(ids=['b', 'a'])

In [25]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [26]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [27]:
#query
query_vector = [random.random() for _ in range(1536)]
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'c', 'score': 0.868069947, 'values': []},
             {'id': 'e', 'score': 0.761916101, 'values': []},
             {'id': 'a', 'score': 0.758797109, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

**Namespace**

In [29]:
# index.describe_index_stats()

import random

vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [35]:
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
# print(vectors)
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

{'upserted_count': 3}

In [36]:
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('qp')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

{'upserted_count': 2}

In [32]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3},
                'second-namespace': {'vector_count': 5}},
 'total_vector_count': 13}

In [37]:
index.fetch(ids=['x'], namespace='first-namespace')

{'namespace': 'first-namespace',
 'usage': {'read_units': 1},
 'vectors': {'x': {'id': 'x',
                   'values': [0.21003975,
                              0.327429295,
                              0.766222894,
                              0.420219243,
                              0.181233436,
                              0.634140551,
                              0.977937698,
                              0.247173756,
                              0.211794436,
                              0.786594927,
                              0.69936192,
                              0.757444322,
                              0.171877325,
                              0.495840937,
                              0.651834309,
                              0.100720197,
                              0.638737142,
                              0.981351912,
                              0.968736351,
                              0.329117775,
                              0.278019786,
       

In [38]:
index.delete(ids=['x'], namespace='first-namespace')

{}

In [39]:
index.delete(delete_all=True, namespace='first-namespace')

{}

In [40]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'second-namespace': {'vector_count': 7}},
 'total_vector_count': 12}

**Splitting and Embedding Text Using Langhain**

In [41]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv("/content/drive/MyDrive/Udemy/LangChain/p-1/.env", override=True)

True

In [42]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
with open ('/content/drive/MyDrive/Udemy/LangChain/p-1/robot-history.txt') as f:
  test_file = f.read()

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap= 20,
    length_function=len
)

In [44]:
chunks=text_splitter.create_documents([test_file])
# print(chunks[1])
print(chunks[1].page_content)

bird he called "The Pigeon", which was propelled by steam.[13] Taking up the earlier reference in


In [45]:
print(f'Number of chunks is: {len(chunks)}')

Number of chunks is: 121


**Embedding Cost**

In [51]:
!pip install --upgrade tiktoken


In [53]:
def print_embedding_cost(texts):
    import tiktoken

    # Explicitly get the encoding using get_encoding
    enc = tiktoken.get_encoding('cl100k_base')

    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

print_embedding_cost(chunks)


Total Tokens: 2440
Embedding Cost in USD: 0.000976


In [54]:
from langchain.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
vector = embedding.embed_query(chunks[0].page_content)
print(vector)

[-0.008881820998126577, -0.005710217909520797, -0.008088919993144495, -0.023946936367495935, 0.02181476467429992, -0.0017107333412079036, -0.04549518074429953, -0.024746499122629988, 0.009254950718472989, -0.020801984936110768, 0.025919192529432998, 0.006706341409684922, -0.007622508075542115, -0.02691864690467311, 0.0033681624366520336, 0.002745169122495593, -0.0011568688398091236, 0.05109212748081826, 0.011640316414893747, 0.011373795186074882, -0.025665997594885713, 0.007522562731150358, -0.0015233356458503825, -0.029797078038561953, 0.003358167948778985, 0.009115026956927766, 0.017123991047087873, -0.04149735951804891, 0.007242715208059913, -0.012926280994698823, 0.033688287047994855, -0.018949662628650284, -0.02971712027293248, -0.018056814882292632, -0.049199825359220496, -0.016417709557887242, 0.006233265879285481, -0.005623598859400619, 0.019842508512362852, 0.013539280286643487, 0.01591131968879267, 0.008655277720799904, 0.01051426364105745, -0.0006096673691308153, -0.00760918